In [1]:
import os, pickle, random
import numpy as np
import torch

In [2]:
# change the current directory to the father directory of this file
os.chdir(os.path.dirname(os.path.dirname(os.path.realpath('evaluate2.ipynb'))))
print(os.getcwd())

/home/yangshuaiyu6791/RepresentAttack/inversion_sePoint3


In [3]:
from modules.models import sPoints_net, ePoints_net, se_net
from DataLoader.dataloader import TrajDataset
emb_dim = 128
emb_model = 'neutraj'

# load data
with open("./data/{}/{}/train/train_data".format(emb_model, emb_dim), "rb") as f:
    tmp_embs = pickle.load(f)
    train_embs = tmp_embs['embs']
with open("./data/{}/{}/train/train_norm".format(emb_model, emb_dim), "rb") as f:
    train_norms = pickle.load(f)

with open("./data/{}/{}/val/val_data".format(emb_model, emb_dim), "rb") as f:
    tmp_embs = pickle.load(f)
    val_embs = tmp_embs['embs']
with open("./data/{}/{}/val/val_norm".format(emb_model, emb_dim), "rb") as f:
    val_norms = pickle.load(f)

with open("./data/{}/{}/test/test_data".format(emb_model, emb_dim), "rb") as f:
    tmp_embs = pickle.load(f)
    test_embs = tmp_embs['embs']
with open("./data/{}/{}/test/test_norm".format(emb_model, emb_dim), "rb") as f:
    test_norms = pickle.load(f)
    
with open("./rdata/gps_seqs/traingps", "rb") as f:
    train_gps = pickle.load(f)
    train_gps = train_gps[0:60000]
with open("./rdata/gps_seqs/valgps", "rb") as f:
    val_gps = pickle.load(f)
with open("./rdata/gps_seqs/testgps", "rb") as f:
    test_gps = pickle.load(f)

In [4]:
# load pth
# delta = 0.001, emb_model = neutraj, embedding dim = emb_dim, hidden dim = 512

model = se_net(emb_dim, 512)
params = torch.load('./models/d0_001_train_sePoints_{}_{}_{}.pth'.format(emb_model, emb_dim, 512))
model.load_state_dict(params)

<All keys matched successfully>

In [5]:
# calculate the distance between two points with longitude and latitude
def distance(lat1, lon1, lat2, lon2):
    R = 6371.004    # km
    pi = 3.1415926
    dlat = (lat2 - lat1) * pi / 180
    dlon = (lon2 - lon1) * pi / 180
    a = np.sin(dlat / 2) * np.sin(dlat / 2) + np.cos(lat1 * pi / 180) * np.cos(lat2 * pi / 180) * np.sin(dlon / 2) * np.sin(dlon / 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = R * c
    return d

# transform the cell x,y to lon, lat
porto_lat_range = [40.953673,41.307945]
porto_lon_range = [-8.735152,-8.156309]

def pred2gps(pred):
    lon = pred[0] * (porto_lon_range[1] - porto_lon_range[0]) + porto_lon_range[0]
    lat = pred[1] * (porto_lat_range[1] - porto_lat_range[0]) + porto_lat_range[0]
    return [lon, lat]

In [6]:
test_embs_ = np.array(test_embs)    # shape: (test_size, dim)
test_norms_ = np.array([[tnorm[0], tnorm[-1]] for tnorm in test_norms])    # shape: (test_size, 2, 2)
test_gps_ = np.array([[gps[0], gps[-1]] for gps in test_gps])    # shape: (test_size, 2,2)

model.cpu(); model.eval()
test_preds = model(torch.from_numpy(test_embs_).float())
test_preds = test_preds.detach().numpy().reshape(-1, 2, 2)    # shape: (test_size, 4) -- s_lon, s_lat, e_lon, e_lat 

# get the first and last point of every trajectory in test_norms
mean = np.array([(porto_lon_range[1] + porto_lon_range[0])/2, (porto_lat_range[1] + porto_lat_range[0])/2])
std = np.array([(porto_lon_range[1] - porto_lon_range[0])/2, (porto_lat_range[1] - porto_lat_range[0])/2])

test_predgps_ = test_preds * std + mean
test_gps_ = test_gps_ 

# get the gps array
print(test_predgps_.shape)  # pred, shape: (test_size, 2, 2)
print(test_gps_.shape)    # label, shape: (test_size, 2, 2)


(10000, 2, 2)
(10000, 2, 2)


In [30]:
# choose a random index to plot
import folium
my_map = folium.Map(location=[40.953673,-8.735152], zoom_start=12)
choice = np.random.choice(len(test_predgps_))


folium.CircleMarker(location=[test_predgps_[choice][0][1], test_predgps_[choice][0][0]], radius=5, color='blue', fill=True).add_to(my_map)
folium.CircleMarker(location=[test_predgps_[choice][1][1], test_predgps_[choice][1][0]], radius=5, color='blue', fill=True).add_to(my_map)
folium.CircleMarker(location=[test_gps_[choice][0][1], test_gps_[choice][0][0]], radius=5, color='red', fill=True).add_to(my_map)
folium.CircleMarker(location=[test_gps_[choice][1][1], test_gps_[choice][1][0]], radius=5, color='red', fill=True).add_to(my_map)

# plot the real trajectory
gps_val = test_gps[choice][:, [1, 0]]
folium.PolyLine(gps_val, color="red", weight=5, opacity=0.6).add_to(my_map)

my_map   # show the map
# print([val_gps[i][0] for i in choice_list])    # lon, lat
# choice_valgrids = [[val_grids[i][0][0], val_grids[i][0][1], val_grids[i][-1][0], val_grids[i][-1][1]] for i in choice_list]    # x, y
# choice_valembs = [val_embs[i] for i in choice_list]
# choice_valgps = [[val_gps[i][0], val_gps[i][-1]] for i in choice_list]    # [[lon, lat], [lon, lat]]
# # get the prediction
# model.cpu()
# model.eval()
# choice_predgrids = []
# for i in range(len(choice_valembs)):    
#     choice_valembs[i] = torch.Tensor(choice_valembs[i]).unsqueeze(0)
#     choice_predgrids.append(model(choice_valembs[i]).cpu().detach().numpy()[0])

# # for i in range(len(choice_predgrids)):
# #     print(choice_predgrids[i], choice_valgrids[i], '\n')

# # transform the prediction to lon, lat
# choice_predgps = []
# for i in range(len(choice_predgrids)):
#     choice_predgps.append([grid2gps(choice_predgrids[i][0:2]), grid2gps(choice_predgrids[i][2:])])
# print(choice_predgps)

In [8]:
''' not used orz'''

# # calculate the distance between prediction and ground truth
# start_dis = []; end_dis = []
# for i in range(len(choice_predgps)):
#     start_dis.append(distance(choice_predgps[i][0][1], choice_predgps[i][0][0], choice_valgps[i][0][1], choice_valgps[i][0][0]))
#     end_dis.append(distance(choice_predgps[i][1][1], choice_predgps[i][1][0], choice_valgps[i][1][1], choice_valgps[i][1][0]))
# print(start_dis)
# print(end_dis)
# print('average start distance: {} km'.format(np.mean(start_dis)))
# print('average end distance: {} km'.format(np.mean(end_dis)))

# with open("./results/{}_{}_{}_{}_dists".format(delta, emb_model, emb_dim, 512), "w") as f:
#     f.write("choose 40 points by random\n")
#     f.write("choice_list: {}\n".format(choice_list))
#     f.write('average start distance: {} km'.format(np.mean(start_dis)))
#     f.write('average end distance: {} km'.format(np.mean(end_dis)))
#     f.write('\n')
#     f.write('start distance: {}'.format(start_dis))
#     f.write('\n')
#     f.write('end distance: {}'.format(end_dis))

' not used orz'

In [9]:
# # if you want, plot the prediction and ground truth on the map
# import folium
# my_map = folium.Map(location=[40.953673,-8.735152], zoom_start=12)
# choice = np.random.choice(len(choice_predgps))
# print(choice_predgps[choice], choice_valgps[choice])

# folium.CircleMarker(location=[choice_predgps[choice][0][1], choice_predgps[choice][0][0]], radius=5, color='blue', fill=True).add_to(my_map)
# folium.CircleMarker(location=[choice_predgps[choice][1][1], choice_predgps[choice][1][0]], radius=5, color='blue', fill=True).add_to(my_map)
# folium.CircleMarker(location=[choice_valgps[choice][0][1], choice_valgps[choice][0][0]], radius=5, color='red', fill=True).add_to(my_map)
# folium.CircleMarker(location=[choice_valgps[choice][1][1], choice_valgps[choice][1][0]], radius=5, color='red', fill=True).add_to(my_map)

# # plot the real trajectory
# gps_val = val_gps[choice_list[choice]][:, [1, 0]]
# folium.PolyLine(gps_val, color="red", weight=5, opacity=0.6).add_to(my_map)

# my_map   # show the map